https://www.safaribooksonline.com/videos/working-with-big/9781788839068/9781788839068-video2_5
stopped at minute 5.52

In [1]:
import pymongo
from pymongo import MongoClient
client=pymongo.MongoClient('this_mongo_1')

In [2]:
client.database_names()

['admin',
 'config',
 'fitbit',
 'local',
 'packt',
 'packt1',
 'test',
 'test2',
 'test_database',
 'twitter']

In [3]:
client.drop_database('packt')

In [4]:
client.drop_database('packt1')
client.drop_database('test2')

In [5]:
client.database_names()

['admin', 'config', 'fitbit', 'local', 'test', 'test_database', 'twitter']

In [6]:
db=client.packt
testCollection=db.testCollection

In [8]:
import pprint
for doc in testCollection.find():
    pprint.pprint(doc)

In [9]:
cur=testCollection.find()
cur.count()

0

##### insert some documents

In [10]:
testCollection.insert_one({'name': 'alice', 'salary': 50000})
testCollection.insert_one({'name': 'bob', 'salary': 40000})
testCollection.insert_one({'name': 'charlie', 'salary': 60000})

In [11]:
cur.count()

3

In [12]:
cur.close()

### set and indvidual document 

In [13]:
res = testCollection.update_one({'name': 'alice'}, {'$set': {'salary': 55000}})

In [14]:
testCollection.find_one({'name': 'alice'}, {'_id': 0})

{'name': 'alice', 'salary': 55000}

#### remove field from docuemnt

In [18]:
res = testCollection.update_one({'name': 'alice'}, {'$unset': {'salary': ""}})

In [19]:
res.acknowledged

True

In [20]:
cur=testCollection.find()
cur.count()

3

In [21]:
testCollection.find_one({'name': 'alice'}, {'_id': 0})

{'name': 'alice'}

### calculate total and mean salary 

In [22]:
pipeline = []

In [23]:
#add docuemnts where salary exists to the pipeline
pipeline.append({"$match": {"salary": {'$exists': "True"}}})

In [24]:
pipeline

[{'$match': {'salary': {'$exists': 'True'}}}]

In [25]:
#creates a new field avSalary and totalSalary with the avg and sum of salary
pipeline.append({'$group':{"_id":None,"avSalary":{"$avg":"$salary"},"totalSalary":{"$sum":"$salary"}}})

In [26]:
cur = testCollection.aggregate(pipeline=pipeline)

In [27]:
cur.next()

{'_id': None, 'avSalary': 50000.0, 'totalSalary': 100000}

In [28]:
for d in testCollection.find():
    print(d)

{'_id': ObjectId('5b45187e0d03a0009675d6f2'), 'name': 'alice'}
{'_id': ObjectId('5b45187e0d03a0009675d6f3'), 'name': 'bob', 'salary': 40000}
{'_id': ObjectId('5b45187e0d03a0009675d6f4'), 'name': 'charlie', 'salary': 60000}


In [29]:
testCollection.drop()

In [34]:
testCollection.insert_one({'name': 'alice', 'salary': 50000, 'unit': 'legal'})
testCollection.insert_one({'name': 'bob', 'salary': 40000, 'unit': 'marketing'})
testCollection.insert_one({'name': 'charlie', 'salary': 60000, 'unit': 'communications'})
testCollection.insert_one({'name': 'david', 'salary': 70000, 'unit': 'legal'})
testCollection.insert_one({'name': 'edwina', 'salary': 90000, 'unit': 'communications'})

In [35]:
pipeline = []
pipeline.append({'$group':{"_id":"$unit","avSalary":{"$avg":"$salary"},"totalSalary":{"$sum":"$salary"}}})

In [36]:
cur = testCollection.aggregate(pipeline=pipeline)

In [37]:
for d in cur:
    print(d)

{'_id': 'communciations', 'avSalary': 90000.0, 'totalSalary': 90000}
{'_id': 'communications', 'avSalary': 70000.0, 'totalSalary': 210000}
{'_id': 'marketing', 'avSalary': 40000.0, 'totalSalary': 80000}
{'_id': 'legal', 'avSalary': 60000.0, 'totalSalary': 240000}


#### match by condition on array index

In [14]:
#find in key ages and secnd position 
coll.find_one({'ages.1' : 42},{'_id': 0 } )

{'ages': [30, 42, 12, 19]}

In [16]:
#find in key ages and secnd position gt 40
coll.find_one({'ages.1' : {'$gt' : 40}},{'_id': 0 } )

{'ages': [30, 42, 12, 19]}

#### insert documents with nested dictionary

In [18]:
res = coll.insert_one({'personInfo': {'name': 'alice', 'age': 30}})
res = coll.insert_one({'personInfo': {'name': 'bob', 'age': 42}})
res = coll.insert_one({'personInfo': {'name': 'charlie', 'age': 12}})
res = coll.insert_one({'personInfo': {'name': 'david', 'age': 19}})

#### match name

In [19]:
coll.find_one({'personInfo.name': 'alice'}, {'_id': 0})

{'personInfo': {'name': 'alice', 'age': 30}}

#### match by age

In [21]:
coll.find_one({'personInfo.age': {'$gt': 40}}, {'_id': 0})

{'personInfo': {'name': 'bob', 'age': 42}}